In [1]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embeddings = model.encode(sentences)
print(embeddings)


/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[[-0.00904849  0.01913359  0.02657794 ... -0.00876412 -0.05602206
  -0.02163094]
 [-0.00785422  0.03018173  0.03129675 ... -0.01246878 -0.06240674
  -0.0059097 ]]


In [3]:
import pandas as pd
df_meta = pd.read_pickle('/home/nick/RQ-VAE-Recommender/dataset/beauty/processed/metadata_sentence.pkl')

In [9]:
import pickle
from tqdm import tqdm
embeddings_dict = {}
for i, row in tqdm(df_meta[['asin', 'sentence']].iterrows()):
    embeddings_dict[row['asin']] = model.encode(row['sentence'])

with open('/home/nick/RQ-VAE-Recommender/dataset/beauty/processed/embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings_dict, f)

259204it [18:38, 231.64it/s]


In [23]:
import chardet

# 檢測編碼
with open('dataset/ml-1m/raw/movies.dat', 'rb') as file:
    raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']

print(f"Detected encoding: {encoding}")

Detected encoding: ISO-8859-1


In [24]:
import pickle
from tqdm import tqdm
embeddings_dict = {}
with open('dataset/ml-1m/raw/movies.dat', 'rb') as file:
    lines = file.readlines()
    for line in tqdm(lines):
        line = line.decode('ISO-8859-1').strip().split('::')
        movie_id = line[0]
        movie_name = ''.join(line[1].split(' (')[:-1])
        movie_year = line[1].split(' (')[-1].strip(')')
        movie_cat = line[2].split('|')
        movie_cat_str = ', '.join(movie_cat)
        # sentence = f"[{movie_name}] is a movie released in {movie_year}. categories: {movie_cat_str}."
        sentence = f'The movie titled "{movie_name}", released in the year {movie_year}, falls under the categories of {movie_cat_str}'
        embeddings = model.encode(sentence)
        embeddings_dict[int(movie_id)] = embeddings

with open('dataset/ml-1m/processed/embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings_dict, f)

100%|██████████| 3883/3883 [00:19<00:00, 203.79it/s]


In [2]:
from data.movie_lens import MovieLensMovieData_from_embeddings
dataset = MovieLensMovieData_from_embeddings('dataset/ml-1m')
print(len(dataset[0]))

768


In [4]:
from modules.rqvae import RqVae
import torch
model = RqVae(
        input_dim=768,
        embed_dim=96,
        hidden_dim=32,
        codebook_size=18,
        n_layers=3
    )
model.load_state_dict(torch.load('model/best_model.pth'))
model.eval()
tensor = torch.tensor(dataset[100]).unsqueeze(0)
output = model(tensor, 0.1, inference_only=True)
print(output)

RqVaeOutput(embeddings=tensor([[[ 8.6342e-01, -8.1423e-02, -5.5795e-01],
         [ 3.8336e-01, -5.3591e-02, -1.3050e-01],
         [ 1.7875e-01,  1.0427e-01,  6.8338e-01],
         [-5.3907e-01,  2.1487e-02,  1.1842e+00],
         [-1.8409e+00, -6.0062e-02,  1.3555e+00],
         [-9.8929e-01, -2.0820e-02, -7.3043e-01],
         [ 6.1163e-01, -2.0457e-01,  5.7366e-01],
         [-2.7220e-01, -1.8315e-01, -9.2454e-01],
         [-1.0223e+00, -5.5717e-02, -1.8583e+00],
         [ 1.5896e+00, -7.3189e-02, -8.0277e-01],
         [-3.7267e-01,  3.2768e-01,  5.6650e-01],
         [ 1.4416e-01, -6.8966e-02,  8.4322e-01],
         [-1.0863e+00, -5.2032e-03, -3.9928e-01],
         [-4.1238e-01,  9.9697e-02,  7.1688e-01],
         [-8.0035e-01,  1.1760e-01, -9.7424e-01],
         [ 3.8582e-01, -8.7549e-03,  1.4761e+00],
         [-1.1912e+00,  5.0613e-02,  4.5632e-01],
         [ 3.5895e-01, -1.9070e-02, -2.6142e-01],
         [-1.3698e-01, -3.6141e-01,  1.5088e-01],
         [-2.1422e-01, -5.2

/tmp/ipykernel_3095885/2064254168.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(dataset[100]).unsqueeze(0)


In [12]:
from torch.utils.data import DataLoader
from modules.rqvae import RqVae
import torch
from data.movie_lens import MovieLensMovieData_from_embeddings
dataset = MovieLensMovieData_from_embeddings('dataset/ml-1m')
model = RqVae(
        input_dim=768,
        embed_dim=32,
        hidden_dim=32,
        codebook_size=256,
        n_layers=3
    )
model.load_state_dict(torch.load('model/best_model.pth'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model.to(device)
model.eval()
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
for iter, data in enumerate(dataloader):
    data = data.to(device)
    output = model(data, 0.1, inference_only=True)
    print(output.sem_ids)
    break

Using device: cuda
tensor([[224,  30,  99]], device='cuda:0')


In [44]:
x = dataset[10].unsqueeze(0).to(device)
output = model(x, 0.1, inference_only=True)
output

RqVaeOutput(embeddings=tensor([[[-2.1279e-01, -1.6085e-03, -1.0829e-03],
         [-1.5489e-01, -2.2287e-03, -2.4117e-03],
         [-2.8902e-01, -2.2383e-03,  3.3424e-04],
         [ 2.3183e-01,  2.9849e-03,  4.6207e-03],
         [ 1.5102e-02, -2.9136e-04, -4.9939e-03],
         [-1.9096e-01, -4.1932e-03, -4.5743e-03],
         [-4.7550e-02, -1.2259e-03, -2.2245e-03],
         [-7.2962e-02,  2.1098e-04,  1.4350e-03],
         [-5.4398e-02, -8.1457e-04, -3.1861e-03],
         [-2.7724e-02,  7.2740e-04, -1.9347e-03],
         [-3.1224e-02, -2.7112e-03, -3.2395e-03],
         [ 8.1348e-02,  1.9481e-03,  1.6046e-06],
         [ 2.1638e-02, -1.0291e-03, -3.9482e-03],
         [ 1.0481e-02,  1.3886e-03, -2.9176e-03],
         [ 5.6225e-02,  2.8352e-03,  3.1070e-03],
         [-4.0707e-02, -2.1325e-03, -5.5716e-03],
         [ 1.7387e-01, -1.8574e-03, -6.8113e-04],
         [ 8.1570e-02,  4.0610e-04, -4.0925e-04],
         [-8.4180e-02, -2.9425e-03, -2.0990e-03],
         [ 1.4722e-01, -1.2

In [29]:
max = 0
min = 1
for i in range(len(dataset)):
    for j in range(len(dataset)):
        if i == j:
            continue
        a = dataset[i]
        b = dataset[j]
        sim = torch.dot(a, b)/(torch.norm(a)*torch.norm(b))
        if sim > max:
            max = sim
        if sim < min:
            min = sim
print(max, min)


tensor(0.9619) tensor(0.8055)


In [28]:
i = 0
j = 900
a = dataset[i]
b = dataset[j]
sim = torch.dot(a, b)/(torch.norm(a)*torch.norm(b))
print(sim)

tensor(0.7803)
